# 🏗️ 실제 운영 단계에서 챗봇을 어떻게 관리할까?

---

## 🎯 강의 목표

- 실습에서 만든 챗봇을 **실제 서비스 환경**에 올릴 때 필요한 운영 전략을 배웁니다.
- 단순히 만들고 끝나는 챗봇이 아닌,  
  **지속적으로 개선, 최적화, 관리 가능한 챗봇**으로 발전시키는 방향을 제시합니다.

---

## 🧩 1. 응답 속도와 서버 부담 줄이기: 캐싱 전략

### ✅ 왜 캐싱이 필요한가?

- 동일하거나 유사한 질문이 반복될 때, 매번 LLM 호출은 비효율적.
- LLM API 비용, 호출 속도, 서버 부하 모두 문제 발생.

---

### 💡 캐싱 전략 예시

| 캐싱 대상 | 설명 |
|-----------|------|
| 질문-응답 쌍 | 동일 질문에 대해 이전 답변 재사용 |
| 검색 결과 | 동일 쿼리의 문서 Top-k 결과 캐싱 |
| 프롬프트 출력 | 자주 쓰이는 프롬프트 형식 저장 |

---

### 🔧 구현 방법

- Redis, Memcached 같은 인메모리 캐시 사용
- 캐시 Key: 해시화된 `query + user_id`
- TTL(Time-to-Live) 설정해 **신선도 유지**

```python
import redis
cache = redis.Redis()

key = f"{user_id}:{hash(query)}"
if cache.exists(key):
    response = cache.get(key)
else:
    response = run_rag(query)
    cache.setex(key, 3600, response)  # 1시간 유효
````

---

## 🕵️ 2. 챗봇 개선을 위한 로그 관리

### ✅ 왜 로그가 중요한가?

* 사용자가 어떤 질문을 했는지, 어떤 답변이 잘못됐는지 모르면 개선할 수 없음.
* 특히 **답변 실패(미검색)**, **FAQ 미포함 질문**을 파악해야 품질 개선 가능.

---

### 💡 핵심 로그 항목

| 로그 항목              | 설명             |
| ------------------ | -------------- |
| user\_id           | 누가 질문했는가       |
| query              | 어떤 질문이었는가      |
| retrieved\_docs    | 어떤 문서가 검색됐는가   |
| final\_answer      | 어떤 답을 줬는가      |
| feedback(optional) | 사용자가 답변에 만족했는가 |

---

### 📊 로그 활용 방안

* 자주 실패하는 질문 분석 → 문서 보강
* 특정 문서가 지나치게 자주 선택됨 → 임베딩 점검
* 사용자 피드백 수집 → 답변 스타일 개선

---

## 🧪 3. A/B 테스트로 지속적인 개선하기

### ✅ 왜 A/B 테스트가 필요한가?

* 프롬프트, 검색 방식, Re-ranking 전략 등은
  사전 실험 없이는 “어떤 방식이 좋은지” 알 수 없음.

---

### 💡 실전 A/B 테스트 설계

| 비교 대상         | 예시                            |
| ------------- | ----------------------------- |
| 프롬프트 버전       | 기본 vs 근거 강조 vs 요약형            |
| Re-ranking 유무 | 기본 검색 vs Cross-Encoder rerank |
| Chunking 방식   | Fixed vs Sliding window       |

---

### 🔧 A/B 실험 구성 방법

1. 사용자 그룹을 랜덤 분리 (예: A/B 그룹)
2. 같은 질문에 대해 서로 다른 처리 방식 적용
3. 로그 수집 후 비교 (예: 답변 채택률, 피드백 점수)

---

## 📌 마무리 요약

| 전략      | 목적         |
| ------- | ---------- |
| 응답 캐싱   | 속도 ↑, 비용 ↓ |
| 로그 관리   | 개선 포인트 발견  |
| A/B 테스트 | 최적 전략 검증   |
